# Загрузка данных.

In [1]:
import os
import pandas as pd

In [2]:
main_dir = os.getcwd()

In [3]:
tab_test = pd.read_csv(main_dir + '/data/NN_Data_1_Test.csv', sep = ',')
tab_train = pd.read_csv(main_dir + '/data/NN_Data_1_Tr_Val.csv', sep = ',')
tab_test = tab_test[['X', 'Y']].drop_duplicates()
tab_train = tab_train[['X', 'Y']].drop_duplicates()

In [4]:
tab_old = pd.read_csv(main_dir + '/data/NN_Data.csv')

In [5]:
len(tab_train), len(tab_test), len(tab_old)

(1991548, 551547, 2543093)

In [6]:
tab_train = pd.merge(tab_train, tab_old, 'inner', left_on = ['X', 'Y'], right_on = ['X', 'Y'])
tab_test = pd.merge(tab_test, tab_old, 'inner', left_on = ['X', 'Y'], right_on = ['X', 'Y'])

In [7]:
len(tab_train), len(tab_test)

(1991547, 551546)

In [8]:
y_train = tab_train[['X', 'Y', 'Dep_labels', 'F', 'Intr', 'MF']]
y_test = tab_test[['X', 'Y', 'Dep_labels', 'F', 'Intr', 'MF']]

In [9]:
X_train = tab_train.copy()
X_test = tab_test.copy()
X_train.drop(['Dep_labels', 'F', 'Intr', 'MF', 'Line'], axis = 1, inplace = True)
X_test.drop(['Dep_labels', 'F', 'Intr', 'MF', 'Line'], axis = 1, inplace = True)

In [10]:
import random

In [11]:
random.seed(239)

In [12]:
X_train_train = X_train.copy()
X_train_val = X_test.copy()
y_train_train = y_train.copy()
y_train_val = y_test.copy()

In [13]:
len(X_train_train), len(X_train_val), len(y_train_train), len(y_train_val)

(1991547, 551546, 1991547, 551546)

In [14]:
for col in y_train_train.columns:
    y_train_train.loc[pd.isna(y_train_train[col]), col] = y_train_train[col].median()
    y_train_val.loc[pd.isna(y_train_val[col]), col] = y_train_val[col].median()

# Модель для Intr, номер 1

In [15]:
X_train.columns

Index(['X', 'Y', 'ARK_0', 'ARK_5000', 'Class_GM_16', 'Dyke_density', 'f4_AuAs',
       'f7_AuAgAs_WCuMo', 'Fault_dist', 'Faults_100', 'Faults_250', 'Gr',
       'Gr_4_5', 'Gr_loc', 'Intr_dist', 'K', 'MED', 'MFault_dist', 'Mg',
       'Mg_loc', 'Mg_tilt', 'Th', 'U'],
      dtype='object')

In [16]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train.drop(['Intr_dist', 'X', 'Y'], axis = 1, inplace = True)
y_train = y_train[['Intr']]

In [17]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val.drop(['Intr_dist', 'X', 'Y'], axis = 1, inplace = True)
y_val = y_val[['Intr']]

In [18]:
from catboost import CatBoostClassifier

C:\playground\Anaconda\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [19]:
def calculate_iou(res_frame, col_target, col_pred):
    return len(res_frame[(res_frame[col_target] == 1) & (res_frame[col_pred] == 1)])/len(res_frame[(res_frame[col_target] == 1) | (res_frame[col_pred] == 1)])

In [20]:
iterations = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 250]

In [21]:
#ious = []
#for iteration in iterations:
#    print(iteration)
#    model = CatBoostClassifier(iterations=iteration, verbose = False)
#    model.fit(X_train, y_train)
#    res_frame = pd.DataFrame(y_val.values, columns=['target'])
#    res_frame['pred'] = model.predict(X_val)
#    iou = calculate_iou(res_frame, 'target', 'pred')
#    print(iou)
#    ious.append(iou)

# we stop at 200 with 61.17

In [22]:
model = CatBoostClassifier(iterations=200, verbose=False)
model.fit(X_train, y_train)

In [23]:
res_frame = pd.DataFrame(y_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_val)
calculate_iou(res_frame, 'target', 'pred')

0.6117160162702713

In [26]:
res_importances = pd.DataFrame({'importances': model.feature_importances_, 'names': model.feature_names_})

In [30]:
res_importances.sort_values(['importances'], ascending = False).head(5)

,importances,names
9,18.474198,Gr
14,16.617901,MFault_dist
3,11.136898,Dyke_density
10,10.690835,Gr_4_5
5,10.076821,f7_AuAgAs_WCuMo


In [33]:
#model.save_model('catboost_for_intr_no_coords_1')

# Модель для Intr, номер 2.

In [35]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train = X_train[['Gr_4_5', 'K', 'MED']]
y_train = y_train[['Intr']]

In [36]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val = X_val[['Gr_4_5', 'K', 'MED']]
y_val = y_val[['Intr']]

In [38]:
ious = []
for iteration in iterations:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

5
0.46536970672127553
10
0.45525029188558086
20
0.4574933469432394
30
0.454196808317278
40
0.4491600674120653
50
0.45090823392915413
60
0.45403387262594014
70
0.4539600656553814
80
0.4538763404886341
90
0.4532825089424432
100
0.4522807334477464
150
0.45257418437674324
200
0.45163557718302244
250
0.4484449073408993


# we stop at 5 with iou 46.54

In [39]:
model = CatBoostClassifier(iterations=5, verbose=False)
model.fit(X_train, y_train)

In [40]:
res_frame = pd.DataFrame(y_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_val)
calculate_iou(res_frame, 'target', 'pred')

0.46536970672127553

In [41]:
model.save_model('catboost_for_intr_no_coords_2')

In [44]:
tab2 = pd.read_csv(main_dir + '/data/NN_Data_2.csv')

# Модель для Intr, номер 4

In [44]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train.drop(['X', 'Y', 'Intr_dist'], axis = 1, inplace = True)
cols = [col for col in X_train.columns if col in tab2.columns]
X_train = X_train[cols]
y_train = y_train[['Intr']]

In [45]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val.drop(['X', 'Y', 'Intr_dist'], axis = 1, inplace = True)
cols = [col for col in X_val.columns if col in tab2.columns]
X_val = X_val[cols]
y_val = y_val[['Intr']]

In [46]:
ious = []
for iteration in iterations:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

5
0.40386029712193766
10
0.487277629703093
20
0.5109789057877145
30
0.4948326464343699
40
0.49318711574863505
50
0.49322667840080464
60
0.49753571260932833
70
0.502216860674615
80
0.5082454245043213
90
0.5022386883178307
100
0.5105152885206131
150
0.5249979898689394
200
0.5740889978976874
250
0.5855892494032358


In [47]:
iterations_2 = [300, 350, 400, 450, 500, 550, 600]

In [48]:
ious = []
for iteration in iterations_2:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

300
0.5916247131446463
350
0.5945336764575384
400
0.5891735655152727
450
0.5934182399540139
500
0.5658112582781457
550
0.5722223208421352
600
0.5840171834554166


# We stop at 350 with 59.45%

In [49]:
model = CatBoostClassifier(iterations=350, verbose=False)
model.fit(X_train, y_train)

In [50]:
res_frame = pd.DataFrame(y_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_val)
calculate_iou(res_frame, 'target', 'pred')

0.5945336764575384

In [51]:
model.save_model('catboost_for_intr_no_coords_4')

# Модель для Intr номер 5

In [52]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train = X_train[['Gr_4_5', 'Class_GM_16', 'Gr_loc']]
y_train = y_train[['Intr']]

In [53]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val = X_val[['Gr_4_5', 'Class_GM_16', 'Gr_loc']]
y_val = y_val[['Intr']]

In [54]:
ious = []
for iteration in iterations:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

5
0.2812970336641963
10
0.3055514046622833
20
0.3087997321578037
30
0.3040590679726922
40
0.3050721542261761
50
0.3052093973442288
60
0.30638100537933594
70
0.307638168617573
80
0.3010170240990493
90
0.30020582181711264
100
0.30053079911715885
150
0.2955381533418321
200
0.29192048148823635
250
0.29027267101095783


# we stop at 20 with 30.88%

In [55]:
model = CatBoostClassifier(iterations=20, verbose=False)
model.fit(X_train, y_train)

In [56]:
res_frame = pd.DataFrame(y_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_val)
calculate_iou(res_frame, 'target', 'pred')

0.3087997321578037

In [57]:
model.save_model('catboost_for_intr_no_coords_5')

# Модель для Dep_labels, номер 1.

In [58]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train.drop(['X', 'Y'], axis = 1, inplace = True)
y_train = y_train[['Dep_labels']]

In [59]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val.drop(['X', 'Y'], axis = 1, inplace = True)
y_val = y_val[['Dep_labels']]

In [61]:
ious = []
for iteration in iterations:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

5
0.029072681704260653
10
0.032160059281215264
20
0.033950617283950615
30
0.03884776075643187
40
0.0385930993456276
50
0.03750558118767674
60
0.0385470719051149
70
0.0374757860229474
80
0.03702879192638765
90
0.03715766099185788
100
0.03715766099185788
150
0.038936232735821336
200
0.03939102800975538
250
0.03939102800975538


# We stop at 20 with 3.4%

In [62]:
model = CatBoostClassifier(iterations=20, verbose=False)
model.fit(X_train, y_train)

In [63]:
res_frame = pd.DataFrame(y_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_val)
calculate_iou(res_frame, 'target', 'pred')

0.033950617283950615

In [64]:
model.save_model('catboost_for_dep_labels_no_coords_1')

# Модель для Dep_labels номер 2

In [65]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train = X_train[['f4_AuAs', 'f7_AuAgAs_WCuMo', 'Gr_4_5']]
y_train = y_train[['Dep_labels']]

In [66]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val = X_val[['f4_AuAs', 'f7_AuAgAs_WCuMo', 'Gr_4_5']]
y_val = y_val[['Dep_labels']]

In [67]:
ious = []
for iteration in iterations:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

5
0.0
10
0.004117736769864267
20
0.010771864745327436
30
0.016466370094947214
40
0.01599365540942436
50
0.015343739621388243
60
0.0183007399577167
70
0.017978400051736402
80
0.01840885684860968
90
0.018060286650813034
100
0.0215422434985121
150
0.02179255043973099
200
0.02767467604349808
250
0.02551917192891088


# We stop at 200 with 2.77%

In [68]:
model = CatBoostClassifier(iterations=200, verbose=False)
model.fit(X_train, y_train)

In [69]:
res_frame = pd.DataFrame(y_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_val)
calculate_iou(res_frame, 'target', 'pred')

0.02767467604349808

In [70]:
model.save_model('catboost_for_dep_labels_no_coords_2')

# Модель для F номер 1

In [31]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train.drop(['X', 'Y', 'Fault_dist', 'MFault_dist'], axis = 1, inplace = True)
y_train = y_train[['F']]

In [32]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val.drop(['X', 'Y', 'Fault_dist', 'MFault_dist'], axis = 1, inplace = True)
y_val = y_val[['F']]

In [33]:
#ious = []
#for iteration in iterations:
#    print(iteration)
#    model = CatBoostClassifier(iterations=iteration, verbose=False)
#    model.fit(X_train, y_train)
#    res_frame = pd.DataFrame(y_val.values, columns=['target'])
#    res_frame['pred'] = model.predict(X_val)
#    iou = calculate_iou(res_frame, 'target', 'pred')
#    print(iou)
#    ious.append(iou)

In [34]:
iterations_2 = [300, 350, 400, 450, 500, 550, 600]

In [35]:
#ious_2 = []
#for iteration in iterations_2:
#    print(iteration)
#    model = CatBoostClassifier(iterations=iteration, verbose=False)
#    model.fit(X_train, y_train)
#    res_frame = pd.DataFrame(y_val.values, columns=['target'])
#    res_frame['pred'] = model.predict(X_val)
#    iou = calculate_iou(res_frame, 'target', 'pred')
#    print(iou)
#    ious_2.append(iou)

# We stop at 450 with 52.52%

In [36]:
model = CatBoostClassifier(iterations=450, verbose=False)
model.fit(X_train, y_train)

In [37]:
res_frame = pd.DataFrame(y_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_val)
calculate_iou(res_frame, 'target', 'pred')

0.5251568167811544

In [38]:
res_importances = pd.DataFrame({'importances': model.feature_importances_, 'names': model.feature_names_})

In [39]:
res_importances.sort_values(['importances'], ascending = False).head(5)

,importances,names
7,24.762575,Faults_250
8,12.580639,Gr
9,9.244273,Gr_4_5
4,9.225721,f4_AuAs
3,8.524579,Dyke_density


In [77]:
#model.save_model('catboost_for_F_no_coords_1')

# Модель для F номер 2

In [78]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train = X_train[['Gr', 'Class_GM_16', 'Intr_dist']]
y_train = y_train[['F']]

In [79]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val = X_val[['Gr', 'Class_GM_16', 'Intr_dist']]
y_val = y_val[['F']]

In [80]:
iterations

[5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 250]

In [82]:
ious = []
for iteration in iterations:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

5
0.44585847126058975
10
0.416544078514641
20
0.42767080481911884
30
0.4273576519733218
40
0.42399373912104815
50
0.4249208300430652
60
0.4282853364281666
70
0.42250857619011545
80
0.42470087203407014
90
0.426034471663306
100
0.42713309483515777
150
0.4252392328568651
200
0.41013234055564796
250
0.4125448650379602


# We stop at 5 with 44.59%

In [83]:
model = CatBoostClassifier(iterations=5, verbose=False)
model.fit(X_train, y_train)

In [84]:
res_frame = pd.DataFrame(y_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_val)
calculate_iou(res_frame, 'target', 'pred')

0.44585847126058975

In [85]:
model.save_model('catboost_for_F_no_coords_2')

# Модель для F номер 4.

In [45]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train.drop(['X', 'Y', 'Fault_dist', 'MFault_dist'], axis = 1, inplace = True)
cols = [col for col in X_train.columns if col in tab2.columns]
X_train = X_train[cols]
y_train = y_train[['F']]

In [46]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val.drop(['X', 'Y', 'Fault_dist', 'MFault_dist'], axis = 1, inplace = True)
cols = [col for col in X_val.columns if col in tab2.columns]
X_val = X_val[cols]
y_val = y_val[['F']]

In [47]:
#ious = []
#for iteration in iterations:
#    print(iteration)
#    model = CatBoostClassifier(iterations=iteration, verbose=False)
#    model.fit(X_train, y_train)
#    res_frame = pd.DataFrame(y_val.values, columns=['target'])
#    res_frame['pred'] = model.predict(X_val)
#    iou = calculate_iou(res_frame, 'target', 'pred')
#    print(iou)
#    ious.append(iou)

In [48]:
#ious_2 = []
#for iteration in iterations_2:
#    print(iteration)
#    model = CatBoostClassifier(iterations=iteration, verbose=False)
#    model.fit(X_train, y_train)
#    res_frame = pd.DataFrame(y_val.values, columns=['target'])
#    res_frame['pred'] = model.predict(X_val)
#    iou = calculate_iou(res_frame, 'target', 'pred')
#    print(iou)
#    ious_2.append(iou)

# We stop at 450 with 52.88%

In [49]:
model = CatBoostClassifier(iterations=450, verbose=False)
model.fit(X_train, y_train)

In [50]:
res_frame = pd.DataFrame(y_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_val)
calculate_iou(res_frame, 'target', 'pred')

0.5287583456433593

In [51]:
res_importances = pd.DataFrame({'importances': model.feature_importances_, 'names': model.feature_names_})

In [52]:
res_importances.sort_values(['importances'], ascending = False).head(5)

,importances,names
4,23.880668,Faults_250
5,13.540246,Gr
6,9.960549,Gr_4_5
8,9.293124,Intr_dist
2,9.198119,f4_AuAs


In [94]:
#model.save_model('catboost_for_F_no_coords_4')

# Модель для F номер 5.

In [95]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train = X_train[['Gr', 'Class_GM_16', 'Mg']]
y_train = y_train[['F']]

In [96]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val = X_val[['Gr', 'Class_GM_16', 'Mg']]
y_val = y_val[['F']]

In [97]:
ious = []
for iteration in iterations:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

5
0.39192336758518787
10
0.37274469149121475
20
0.3857802278085877
30
0.37434230154724696
40
0.3785136505685332
50
0.38388006101737465
60
0.3869834879355839
70
0.39059607552758235
80
0.40223005833953396
90
0.39961529213753305
100
0.4005532481017587
150
0.41252861075090247
200
0.4244836662522002
250
0.42194961924020136


# We stop at 200 iterations with 42.45%

In [98]:
model = CatBoostClassifier(iterations=200, verbose=False)
model.fit(X_train, y_train)

In [99]:
res_frame = pd.DataFrame(y_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_val)
calculate_iou(res_frame, 'target', 'pred')

0.4244836662522002

In [100]:
model.save_model('catboost_for_F_no_coords_5')

# Модель для MF номер 1

In [101]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train.drop(['X', 'Y', 'Fault_dist', 'MFault_dist', 'Gr', "Gr_4_5"], axis = 1, inplace = True)
y_train = y_train[['MF']]

In [102]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val.drop(['X', 'Y', 'Fault_dist', 'MFault_dist', 'Gr', "Gr_4_5"], axis = 1, inplace = True)
y_val = y_val[['MF']]

In [103]:
ious = []
for iteration in iterations:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

5
0.11241278846891052
10
0.12005275045095572
20
0.1381417554168197
30
0.1455496776804556
40
0.15853260946602973
50
0.17069311252677097
60
0.17606526429341962
70
0.1990867824480122
80
0.20043547110055424
90
0.1973308165733959
100
0.2146364849631367
150
0.22534129037215378
200
0.2232298613087211
250
0.23448327063947733


# We stop at 150 with 22.53%

In [104]:
model = CatBoostClassifier(iterations=150, verbose=False)
model.fit(X_train, y_train)

In [105]:
res_frame = pd.DataFrame(y_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_val)
calculate_iou(res_frame, 'target', 'pred')

0.22534129037215378

In [106]:
model.save_model('catboost_for_MF_no_coords_1')

# Модель для MF номер 2.

In [116]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train.drop(['X', 'Y', 'Fault_dist', 'MFault_dist'], axis = 1, inplace = True)
y_train = y_train[['MF']]

In [117]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val.drop(['X', 'Y', 'Fault_dist', 'MFault_dist'], axis = 1, inplace = True)
y_val = y_val[['MF']]

In [119]:
ious = []
for iteration in iterations:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

5
0.11337254056717061
10
0.1331741762903945
20
0.16954283984256735
30
0.19347010536481485
40
0.2261730540565261
50
0.22580681599963853
60
0.2492987783676024
70
0.24713649659903902
80
0.2491513837159964
90
0.24755362655639473
100
0.2514933288673031
150
0.28280791378190046
200
0.30460624071322434
250
0.3054033432753


In [120]:
ious = []
for iteration in iterations_2:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

300
0.301225735791019
350
0.3025836758661186
400
0.30561382008437227
450
0.30916869548774434
500
0.327864165765497
550
0.3232417098699012
600
0.3331400298592175


In [121]:
iterations_3 = [700, 800, 900, 1000]

In [122]:
ious = []
for iteration in iterations_3:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

700
0.3313990344760326
800
0.3345770824219655
900
0.3194480668678664
1000
0.3266058581706064


# We stop at 800 with 33.46%

In [123]:
model = CatBoostClassifier(iterations=800, verbose=False)
model.fit(X_train, y_train)

In [124]:
res_frame = pd.DataFrame(y_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_val)
calculate_iou(res_frame, 'target', 'pred')

0.3345770824219655

In [125]:
model.save_model('catboost_for_MF_no_coords_2')

In [118]:
_ = [print(col, y_train['MF'].corr(X_train[col])) for col in X_train.columns]

ARK_0 0.07399458852562178
ARK_5000 0.01389769507757192
Class_GM_16 0.06527319010824902
Dyke_density 0.13766651460898732
f4_AuAs 0.028388023975957806
f7_AuAgAs_WCuMo 0.0036767658672967334
Faults_100 0.06022988195384815
Faults_250 0.15612103988238002
Gr -0.059062618774570246
Gr_4_5 0.05788189363804702
Gr_loc -0.027218412100837724
Intr_dist -0.009264380110663498
K -0.10237502128180441
MED -0.09830391153358019
Mg 0.06768552892123836
Mg_loc 0.01027964657311839
Mg_tilt 0.010225794132475857
Th -0.0784916628290655
U -0.06068433204506312


In [111]:
# Dyke_density 0.13766651460898732
# K -0.10237502128180441
# MED -0.09830391153358019

In [112]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train = X_train[['Dyke_density', 'K', 'MED']]
y_train = y_train[['MF']]

In [113]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val = X_val[['Dyke_density', 'K', 'MED']]
y_val = y_val[['MF']]

In [114]:
ious = []
for iteration in iterations:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

5
0.0
10
0.0
20
0.0
30
0.0
40
0.0
50
0.0
60
0.0
70
1.597724839828085e-05
80
1.5978780179921066e-05
90
1.5976482617586914e-05
100
0.0
150
0.00011165164686179121
200
0.00012740882306099699
250
0.00012735811509989652


# Модель для MF номер 3.

In [126]:
X_train = X_train_train.copy()
y_train = y_train_train.copy()
X_train.drop(['X', 'Y', 'Fault_dist', 'MFault_dist', 'Gr', 'Gr_4_5'], axis = 1, inplace = True)
cols = [col for col in X_train.columns if col in tab2.columns]
X_train = X_train[cols]
y_train = y_train[['MF']]

In [127]:
X_val = X_train_val.copy()
y_val = y_train_val.copy()
X_val.drop(['X', 'Y', 'Fault_dist', 'MFault_dist', 'Gr', 'Gr_4_5'], axis = 1, inplace = True)
cols = [col for col in X_val.columns if col in tab2.columns]
X_val = X_val[cols]
y_val = y_val[['MF']]

In [128]:
ious = []
for iteration in iterations:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

5
0.1109736969111969
10
0.15073558860438416
20
0.1550530874357487
30
0.14607884351988717
40
0.16377617576381737
50
0.1738692783785622
60
0.17786432976306393
70
0.18346099098488444
80
0.1908931462493254
90
0.1985696368051664
100
0.20513303128487115
150
0.22357136350030585
200
0.22130266826984574
250
0.23000038004028428


In [129]:
ious = []
for iteration in iterations_2:
    print(iteration)
    model = CatBoostClassifier(iterations=iteration, verbose=False)
    model.fit(X_train, y_train)
    res_frame = pd.DataFrame(y_val.values, columns=['target'])
    res_frame['pred'] = model.predict(X_val)
    iou = calculate_iou(res_frame, 'target', 'pred')
    print(iou)
    ious.append(iou)

300
0.23319783882044032
350
0.23541854410733953
400
0.23544472194681157
450
0.240425
500
0.25017872505422334
550
0.2547079108285154
600
0.25266061779481924


# We stop at 550 with 25.47%

In [130]:
model = CatBoostClassifier(iterations=550, verbose=False)
model.fit(X_train, y_train)

In [131]:
res_frame = pd.DataFrame(y_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_val)
calculate_iou(res_frame, 'target', 'pred')

0.2547079108285154

In [132]:
model.save_model('catboost_for_MF_no_coords_3')

# Модель для atk_outline

In [53]:
tab1 = pd.read_csv(os.getcwd() + '/data/' + 'NN_Data_1.csv')

In [54]:
X_train_train = X_train_train[['X', 'Y']]
X_train_val = X_train_val[['X', 'Y']]

In [55]:
X_train_train = pd.merge(tab1, X_train_train, 'inner', left_on=['X', 'Y'], right_on=['X', 'Y'], suffixes=['', '_new'])
X_train_val = pd.merge(tab1, X_train_train, 'inner', left_on=['X', 'Y'], right_on=['X', 'Y'], suffixes=['', '_new'])

In [56]:
y_train_train = X_train_train[['Atk_outline']]
y_train_val = X_train_val[['Atk_outline']]

In [57]:
X_train_train.drop(['X', 'Y', 'Line', 'Intr', 'Dep_labels', 'F', 'Faults_100', 'Faults_250', 'MF', 'Atk_outline'], axis = 1, inplace = True)

In [58]:
X_train_val.drop(['Line'], axis = 1, inplace = True)

In [59]:
X_train_val = X_train_val[X_train_train.columns]

In [60]:
#ious = []
#for iteration in iterations:
#    print(iteration)
#    model = CatBoostClassifier(iterations=iteration, verbose=False)
#    model.fit(X_train_train, y_train_train)
#    res_frame = pd.DataFrame(y_train_val.values, columns=['target'])
#    res_frame['pred'] = model.predict(X_train_val)
#    iou = calculate_iou(res_frame, 'target', 'pred')
#    print(iou)
#    ious.append(iou)

In [61]:
#ious_2 = []
#for iteration in iterations_2:
#    print(iteration)
#    model = CatBoostClassifier(iterations=iteration, verbose=False)
#    model.fit(X_train_train, y_train_train)
#    res_frame = pd.DataFrame(y_train_val.values, columns=['target'])
#    res_frame['pred'] = model.predict(X_train_val)
#    iou = calculate_iou(res_frame, 'target', 'pred')
#    print(iou)
#    ious_2.append(iou)

In [62]:
#ious_3 = []
#for iteration in iterations_3:
#    print(iteration)
#    model = CatBoostClassifier(iterations=iteration, verbose=False)
#    model.fit(X_train_train, y_train_train)
#    res_frame = pd.DataFrame(y_train_val.values, columns=['target'])
#    res_frame['pred'] = model.predict(X_train_val)
#    iou = calculate_iou(res_frame, 'target', 'pred')
#    print(iou)
#    ious_3.append(iou)

# We stop at 1000 with 98.14

In [63]:
model = CatBoostClassifier(iterations=1000, verbose=False)
model.fit(X_train_train, y_train_train)

In [64]:
res_frame = pd.DataFrame(y_train_val.values, columns=['target'])
res_frame['pred'] = model.predict(X_train_val)
calculate_iou(res_frame, 'target', 'pred')

0.9813593951989963

In [65]:
res_importances = pd.DataFrame({'importances': model.feature_importances_, 'names': model.feature_names_})

In [66]:
res_importances.sort_values(['importances'], ascending = False).head(5)

,importances,names
8,17.498131,Gr
14,13.560414,MFault_dist
9,10.732923,Gr_4_5
11,10.121163,Intr_dist
5,8.833289,f4_AuAs


In [147]:
#model.save_model('catboost_for_atk_outline_no_coords')